In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import csv
from analysis import *

## Reconstruct graph

In [2]:
ripple_graph_base, caida_graph_base, _ = build_graphs('../Ripple/gateway_links.csv', '../Caida/20190301.as-rel2.txt')

## Add locations

In [3]:
locations = pd.read_csv('locations.txt',delimiter='\n', comment='#', header=None, encoding='ISO-8859-1')

In [4]:
locations['asn'] = locations[0].str.split('|').str[0]
locations['org'] = locations[0].str.split('|').str[3]

In [5]:
asn_org = locations[['asn','org']].set_index('asn').to_dict()['org']

In [6]:
organisations = pd.read_csv('organisations.txt',delimiter='\n', comment='#', header=None, encoding='ISO-8859-1')

In [7]:
organisations['org'] = organisations[0].str.split('|').str[0]
organisations['country'] = organisations[0].str.split('|').str[3]

In [ ]:
org_country = organisations[['org','country']].set_index('org').to_dict()['country']

In [9]:
asn_country = {}
for n in ripple_graph_base.nodes:
    organisation = asn_org[n]
    country = org_country[organisation]
    asn_country[n] = country

In [10]:
countries_data = pd.read_csv('../Ripple/country.csv',delimiter=',', comment='#', encoding='ISO-8859-1')

In [11]:
countries = countries_data[['ISO 3166 Country Code', 'Latitude', 'Longitude']]

In [12]:
countries['lat-lon'] = list(zip(countries.Latitude,countries.Longitude))

In [13]:
countries_dict = countries[['ISO 3166 Country Code', 'lat-lon']].set_index('ISO 3166 Country Code').to_dict()['lat-lon']

In [14]:
asn_lat = {}
asn_lon = {}
for key,value in asn_country.items():
    asn_lat[key] = countries_dict[value][0]
    asn_lon[key] = countries_dict[value][1]

In [15]:
nx.set_node_attributes(ripple_graph_base, name='lat', values=asn_lat)
nx.set_node_attributes(ripple_graph_base, name='lng', values=asn_lon)

## Save graph as file

In [16]:
nx.write_gexf(ripple_graph_base,'ripple.gexf')